In [21]:
dt = 1

1. Delivery
2. Crane Operation Time - not being ready 8- 10
3. Every unit cost 5000 $ - more than 4 worker
4. only 4 types of pbu - 5, 10, 15 

Equations

$ Y_{warehouse}(t) =  Y_{warehouse}(t-1) + x_{delivered}(t) - x_{load}(t) $


$ Y_{parking}(t) =  Y_{parking}(t-1) + x_{load}(t) - x_{hoist}(t) $


$ Y_{Hoist}(t) =  Y_{Hoist}(t-1) + x_{hoist}(t) - x_{install}(t) $


$ Y_{install}(t) =  Y_{install}(t-1) + x_{install}(t) $


$Y_{parked}(t)$ =< 30


In [1]:
import pandas as pd 
import datetime
import numpy as np
from ortools.linear_solver import pywraplp


## Setting up the optimization

In [2]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')
 
 

## Generate Random delivery Data

In [3]:
delivery=pd.DataFrame([])
delivery["date"]=pd.date_range(start='1/1/2022', end='3/31/2022')
delivery["shipment"]=np.random.poisson(10, len(delivery))
delivery["time_index"]=range(1,len(delivery)+1)
n=len(delivery)

In [4]:
delivery.head()

,date,shipment,time_index
0,2022-01-01,6,1
1,2022-01-02,11,2
2,2022-01-03,10,3
3,2022-01-04,10,4
4,2022-01-05,6,5


## Defining Variables

$x_{delivered}(t)$     $~~~~~~~~~~~~~~~~$     PBUs delivered into the warehouse at each time frame

$Y_{warehouse}(t)$     $~~~~~~~~~~~~~~~~$     PBUs in warehouse at each time frame

$x_{load}(t)$     $~~~~~~~~~~~~~~~~$     PBUs loaded from the warehouse into the parking at each time frame

$Y_{parked}(t)$     $~~~~~~~~~~~~~~~~$     PBUs in parking at each time frame

$x_{hoist}(t)$     $~~~~~~~~~~~~~~~~$     PBUs selected from the parking, and and to be hoisted at each time frame

$Y_{hoisted}(t)$     $~~~~~~~~~~~~~~~~$     PBUs which are hoisted at each time frame

$x_{install}(t)$     $~~~~~~~~~~~~~~~~$     PBUs selected from the hoisted ones, and to be installed at each time frame

$Y_{installed}(t)$     $~~~~~~~~~~~~~~~~$     PBUs installed at each time frame



$C_{hoist}$     $~~~~~~~~~~~~~~~~$     Maximum PBUs can be hoisted at each time frame

$C_{install}$     $~~~~~~~~~~~~~~~~$     Maximum PBUs can be installed at each time frame

$C_{parked}$     $~~~~~~~~~~~~~~~~$     Maximum PBUs in parking at each time frame

$C_{load}$     $~~~~~~~~~~~~~~~~$     Maximum PBUs loaded from the warehouse into the parking at each time frame

$C_{installed}$     $~~~~~~~~~~~~~~~~$   Maximum  PBUs installed 





In [19]:
C_hoist= 10
C_install= 8
C_parked= 30
C_load=50
C_installed=44

In [17]:
infinity = solver.infinity()
Y_warehouse={}

for i in range (1,n+1):
    solver.IntVar(lb=0,ub=infinity,name="Y_warehouse_%i" % i)

In [21]:
Y_parked={}

for i in range (1,n+1):
    solver.IntVar(lb=0,ub=C_parked,name="Y_parked_%i" % i)

In [22]:
x_load={}

for i in range (1,n+1):
    solver.IntVar(lb=0,ub=C_load,name="x_load_%i" % i)

In [23]:
x_hoist={}
for i in range (1,n+1):
    solver.IntVar(lb=0,ub=C_hoist,name="x_hoist_%i" % i)

In [24]:
Y_hoisted={}
for i in range (1,n+1):
    solver.IntVar(lb=0,ub=C_hoist,name="Y_hoisted_%i" % i)

In [ ]:
x_install={}
for i in range (1,n+1):
    solver.IntVar(lb=0,ub=C_install,name="x_install_%i" % i)

In [ ]:
Y_installed
for i in range (1,n+1):
    solver.IntVar(lb=0,ub=C_installed,name="Y_installed_%i" % i)

## Defining Mathematical Relationship Between Variables 

The number of the PBUs in the warehouse at time $t$ = The number of the PBUs in the warehouse at time $t-1$  +  the number of PBUs delivered to warehouse at time $t$   -  the number of PBUs loaded from warehouse to parking at time $t$

$ Y_{warehouse}(t) =  Y_{warehouse}(t-1) + x_{delivered}(t) - x_{load}(t) $


$ Y_{parking}(t) =  Y_{parking}(t-1) + x_{load}(t) - x_{hoist}(t) $
